In [1]:
import pandas as pd
from transformers import BertTokenizer, AutoModel
import torch
import torch.nn as nn
import os
from tqdm import tqdm

In [2]:
df = pd.read_csv('C.csv',delimiter=';', encoding='cp1251')
df

,Number,Date,UID,Desc,Group,Cat
0,1346291,01.01.2023 2:10,e87b2cfa-6159-4aa2-b789-3974a01e0b0a,Застряли в лифте,NaN,NaN
1,1346298,01.01.2023 9:09,d6257b7b-d68c-449f-aef3-6c4af1a2d78f,Течь батареи в спальной,NaN,NaN
2,1346301,01.01.2023 10:37,927aeffe-0842-4894-8ada-0a91ccdf1692,Течь сверху по стояку,NaN,NaN
3,1346303,01.01.2023 10:56,9884e4a6-2ce4-485b-913d-8e75c7779302,Течь КНС в 3 подвале,NaN,NaN
4,1346306,01.01.2023 11:44,4593c91a-8b75-46e4-83e5-557d730abea0,Течь кровли (забита ливневка),NaN,NaN
...,...,...,...,...,...,...
28973,1425637,10.09.2023 17:31,e637470f-6fd2-41fc-94af-7c7a83a66b60,"Течь из стыка КНС в подвале, проверить засор в...",NaN,NaN
28974,1425638,10.09.2023 17:50,04e668c0-e11d-44e8-8ee3-f6154b129f86,Нет напора ХВС и совсем пропадает.,NaN,NaN
28975,1425640,10.09.2023 17:54,62f5beff-2753-490c-8115-01cfa320ee40,Обратная тяга вентканала в ванной,NaN,NaN
28976,1425641,10.09.2023 17:55,5f150ce4-dbcc-4cbf-9adb-dde8d4c05aba,Х/ГВС,NaN,NaN


In [3]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [4]:
x_data = df['Desc']
x_tokens = tokenizer.batch_encode_plus(x_data.astype('str'), max_length=10,
                                      padding = 'max_length', truncation = True)
x_seq = torch.tensor(x_tokens['input_ids'])
x_mask = torch.tensor(x_tokens['attention_mask'])

print(x_data.shape, x_seq.shape, x_mask.shape)


(28978,) torch.Size([28978, 10]) torch.Size([28978, 10])


In [5]:
import pickle 

pickle_path = os.path.join('..', 'model', 'encoders.pkl')

with open(pickle_path, "rb") as f:
    encoders = pickle.load(f)

encoders

{'path1': '..\\model\\model1.pt',
 'path2': '..\\model\\model2.pt',
 'encoder1': LabelBinarizer(),
 'encoder2': LabelBinarizer(),
 'num_classes1': 22,
 'num_classes2': 97}

In [6]:
class BERT_Arch(nn.Module):

    def __init__(self):#, bert):
        super(BERT_Arch, self).__init__()
        #self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768,512)
        self.fc2 = nn.Linear(512,encoders['num_classes1'])
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, cls_hs):#, sent_id, mask):
        #cls_hs is pooler output in docs
       #_, cls_hs = self.bert(sent_id, attention_mask = mask, return_dict = False)
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
    
    
    
class second(nn.Module):

    def __init__(self):
        super(second, self).__init__()

        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768 + encoders['num_classes1'],1024)
        self.fc2 = nn.Linear(1024, encoders['num_classes2'])
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, cls_hs, target_1):
        #cls_hs is pooler output in docs
        #_, cls_hs = self.bert(sent_id, attention_mask = mask, return_dict = False)
#         print(cls_hs.shape, target_1.shape)
        concated = (torch.cat((cls_hs, target_1), 1))
        x = self.fc1(concated)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [7]:
model1 = BERT_Arch()
model1.load_state_dict(torch.load(encoders['path1']))
model1.eval()

model2 = second()
model2.load_state_dict(torch.load(encoders['path2']))
model2.eval()


second(
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (fc1): Linear(in_features=790, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=97, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [8]:
for sent_id, mask, num in tqdm(zip((x_seq), x_mask, range((len(x_seq)))), total = len(x_seq)):
    sent_id = sent_id.unsqueeze(0)
    mask = mask.unsqueeze(0)
    with torch.no_grad():
        _, cls_hs = bert(sent_id, mask, return_dict = False)
        pred1 = model1(cls_hs)
        pred2 = model2(cls_hs, pred1)
        
        pred1 = torch.argmax(pred1, dim=1)
        pred2 = torch.argmax(pred2, dim=1)
        
        tmp1 = [0] * encoders['num_classes1']
        tmp2 = [0] * encoders['num_classes2']
        
        tmp1[pred1[0]] = 1
        tmp2[pred2[0]] = 1
        
        tmp1, tmp2 = torch.tensor([tmp1]), torch.tensor([tmp2])
        
        result1 = encoders['encoder1'].inverse_transform(tmp1)
        result2 = encoders['encoder2'].inverse_transform(tmp2)
        
        
        df['Group'][num] = result1[0]
        df['Cat'][num] = result2[0]
        

  0%|                                                                                        | 0/28978 [00:00<?, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_14212\260924398.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Group'][num] = result1[0]
C:\Users\admin\AppData\Local\Temp\ipykernel_14212\260924398.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cat'][num] = result2[0]
100%|████████████████████████████████████████████████████████████████████████████| 28978/28978 [20:19<00:00, 23.76it/s]


In [9]:
df.to_csv(os.path.join('..', 'result', 'A.csv'), encoding='cp1251')

df

,Number,Date,UID,Desc,Group,Cat
0,1346291,01.01.2023 2:10,e87b2cfa-6159-4aa2-b789-3974a01e0b0a,Застряли в лифте,Лифты,ГВС (несоответствие санитарным нормам)
1,1346298,01.01.2023 9:09,d6257b7b-d68c-449f-aef3-6c4af1a2d78f,Течь батареи в спальной,Управление домом,Контрольные показания ХВС\ГВС
2,1346301,01.01.2023 10:37,927aeffe-0842-4894-8ada-0a91ccdf1692,Течь сверху по стояку,Управление домом,ГВС (несоответствие санитарным нормам)
3,1346303,01.01.2023 10:56,9884e4a6-2ce4-485b-913d-8e75c7779302,Течь КНС в 3 подвале,Управление домом,Контрольные показания ХВС\ГВС
4,1346306,01.01.2023 11:44,4593c91a-8b75-46e4-83e5-557d730abea0,Течь кровли (забита ливневка),Управление домом,ГВС (несоответствие санитарным нормам)
...,...,...,...,...,...,...
28973,1425637,10.09.2023 17:31,e637470f-6fd2-41fc-94af-7c7a83a66b60,"Течь из стыка КНС в подвале, проверить засор в...",Управление домом,ГВС (несоответствие санитарным нормам)
28974,1425638,10.09.2023 17:50,04e668c0-e11d-44e8-8ee3-f6154b129f86,Нет напора ХВС и совсем пропадает.,Управление домом,ГВС (несоответствие санитарным нормам)
28975,1425640,10.09.2023 17:54,62f5beff-2753-490c-8115-01cfa320ee40,Обратная тяга вентканала в ванной,Управление домом,ГВС (несоответствие санитарным нормам)
28976,1425641,10.09.2023 17:55,5f150ce4-dbcc-4cbf-9adb-dde8d4c05aba,Х/ГВС,Управление домом,Контрольные показания ХВС\ГВС
